# Fine-Tuning Llama 3 and Using It Locally

 ref: http://gpu015.cluster.local:8888/lab?token=1167d32b2fe5218075a0b1f459da85514cee7742ff3a9b60

## 1. Fine-Tuning Llama 3

In [11]:
# !pip3 install wandb

Object `wandb` not found.


In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch   #, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

### Login to HuggingFace Hub

In [9]:
# from huggingface_hub import login
from huggingface_hub import interpreter_login

interpreter_login()
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()

# hf_token = user_secrets.get_secret("HUGGINGFACE_TOKEN")

# login(token = hf_token)

# wb_token = user_secrets.get_secret("wandb")

# wandb.login(key=wb_token)
# run = wandb.init(
#     project='Fine-tune Llama 3 8B on Medical Dataset', 
#     job_type="training", 
#     anonymous="allow"
# )


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .


Enter your token (input will not be visible):  ········
Add token as git credential? (Y/n)  Y


Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /h/ws_aabboud/.cache/huggingface/token
Login successful


## Set model and dataset parameters

In [32]:
os.chdir("../../../../../../")
os.getcwd()

'/'

In [70]:
base_model = "/fs01/model-weights/Meta-Llama-3-8B-Instruct"
dataset_name = "heliosbrahma/mental_health_conversational_dataset"
new_model = "/h/ws_aabboud/finetuning-and-alignment/Deloitte/finetuned/llama-3-8b-chat-doctor"

'/'

### Set the data type and attention implementation

In [12]:
torch_dtype = torch.float16
attn_implementation = "eager"

## Loading the Model

In [34]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Load Tokenizer

In [35]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)
model, tokenizer = setup_chat_format(model, tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Adding the adapter to the layer

In [36]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

## Loading Dataset

In [59]:
dataset_name = "heliosbrahma/mental_health_conversational_dataset"
# dataset_name = "ruslanmv/ai-medical-chatbot"

In [60]:
dataset = load_dataset(dataset_name, split="all")

In [61]:
dataset["text"][0]

'<<<HUMAN>>>: What is a panic attack? <<<ASSISTANT>>>: Panic attacks come on suddenly and involve intense and often overwhelming fear. They’re accompanied by very challenging physical symptoms, like a racing heartbeat, shortness of breath, or nausea. Unexpected panic attacks occur without an obvious cause. Expected panic attacks are cued by external stressors, like phobias. Panic attacks can happen to anyone, but having more than one may be a sign of panic disorder, a mental health condition characterized by sudden and repeated panic attacks.'

In [62]:
def extract_message(text, tag):
    """
    Extracts a message from the given text based on the specified tag.
    
    Parameters:
    text (str): The text containing the messages.
    tag (str): The tag to search for, either '<<<HUMAN>>>: ' or '<<<ASSISTANT>>>: '.
    
    Returns:
    str: The extracted message, or None if the tag is not found.
    """
    try:
        start_tag = f'<<<{tag}>>>: '
        end_tag = ' <<<'
        
        start_index = text.find(start_tag)
        if start_index == -1:
            return None
        
        start_index += len(start_tag)
        end_index = text.find(end_tag, start_index)
        
        if end_index == -1:
            # If end_tag not found, take the rest of the text
            end_index = len(text)
        
        return text[start_index:end_index].strip()
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [63]:
#Importing the dataset
dataset = load_dataset(dataset_name, split="all")
dataset = dataset.shuffle(seed=65).select(range(100)) # Only use 100 samples for quick demo

def format_chat_template(row):
    row_json = [{"role": "user", "content": extract_message(row["text"], 'HUMAN')},
               {"role": "assistant", "content":  extract_message(row["text"], 'ASSISTANT')}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc=4,
)

dataset['text'][3]

'<|im_start|>user\nWhat are the side effects of medication?<|im_end|>\n<|im_start|>assistant\nLike other medication, psychiatric medication has its own set of side effects like Drowsiness, Restlessness, Dizziness, Dry mouth, Constipation, Nausea, and Vomiting.\nIt is usually because of the body getting used to medication. It normally takes a month for the body to get used to these drugs. If you are feeling any of the above symptoms, call your doctor immediately.<|im_end|>\n'

### Split the Dataset

In [65]:
dataset = dataset.train_test_split(test_size=0.1)

## Setup Training


In [72]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    # report_to="wandb"
)

### set up a supervised fine-tuning (SFT) 

In [73]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


### Train the model

In [74]:
trainer.train()

Step,Training Loss,Validation Loss
9,4.213200,3.842546
18,2.354000,2.851721
27,2.689200,2.467701
36,2.140900,2.232986
45,2.351600,2.175320


TrainOutput(global_step=45, training_loss=3.0031442589230006, metrics={'train_runtime': 55.2579, 'train_samples_per_second': 1.629, 'train_steps_per_second': 0.814, 'total_flos': 552069066129408.0, 'train_loss': 3.0031442589230006, 'epoch': 1.0})

## Model Evaluation

In [82]:

# wandb.finish()
# model.config.use_cache = True

NameError: name 'wandb' is not defined

In [84]:
messages = [
    {
        "role": "user",
        "content": "Hello doctor, I have bad insomnia. How do I get rid of it?"
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, 
                                       add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, 
                   truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=150, 
                         num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])

## Save the Modle Adapter Files

In [86]:
os.getcwd()

'/'

In [87]:
#save to local file
trainer.model.save_pretrained(new_model)


In [91]:
# Push to Huggingface
trainer.model.push_to_hub("ahmabboud/llama-3-8b-chat-doctor")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ahmabboud/llama-3-8b-chat-doctor/commit/5b0f8c2c1374a89458de8069d080ff46ea2f82e4', commit_message='Upload model', commit_description='', oid='5b0f8c2c1374a89458de8069d080ff46ea2f82e4', pr_url=None, pr_revision=None, pr_num=None)

# 2. Merging Llama 3 8B with the adapter

## Merging the base model with the adapter

In [93]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model)

base_model_reload = AutoModelForCausalLM.from_pretrained(
        base_model,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)

# Merge adapter with base model
model = PeftModel.from_pretrained(base_model_reload, new_model)

model = model.merge_and_unload()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## Model Inference

In [96]:
messages = [{"role": "user", "content": "Hello doctor, I have bad insomnia. How do I get rid of it?"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

outputs = pipe(prompt, max_new_tokens=120, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(outputs[0]["generated_text"])

<|im_start|>user
Hello doctor, I have bad insomnia. How do I get rid of it?<|im_end|>
<|im_start|>assistant
Sometimes, we don’t take enough medication. Can we also have a bad appetite? If we have a poor appetite, we don’t eat enough. Do we also have a poor appetite? Sometimes, we eat more than we need. If we eat too much, we also eat more than we need. This causes us to eat more than we need to. Then, we eat more than we need. This causes us to eat more than we need. This causes us to eat more than we need to. Then, we eat more than we need. This causes us to eat more than we need


## Saving and pushing the merged model

In [97]:
finetuned="/h/ws_aabboud/finetuning-and-alignment/Deloitte/finetuned"

In [98]:
model.save_pretrained(f"{finetuned}/llama-3-8b-chat-doctor-merged")
tokenizer.save_pretrained(f"{finetuned}/llama-3-8b-chat-doctor-merged")

('/h/ws_aabboud/finetuning-and-alignment/Deloitte/finetuned/llama-3-8b-chat-doctor-merged/tokenizer_config.json',
 '/h/ws_aabboud/finetuning-and-alignment/Deloitte/finetuned/llama-3-8b-chat-doctor-merged/special_tokens_map.json',
 '/h/ws_aabboud/finetuning-and-alignment/Deloitte/finetuned/llama-3-8b-chat-doctor-merged/tokenizer.json')

# 3. Converting the Model to Llama.cpp GGUF

In [101]:
%cd {finetuned}/llama.cpp
!sed -i 's|MK_LDFLAGS   += -lcuda|MK_LDFLAGS   += -L/usr/local/nvidia/lib64 -lcuda|' Makefile
!LLAMA_CUDA=1 conda run -n base make -j > /dev/null

/fs01/home/ws_aabboud/finetuning-and-alignment/Deloitte/finetuned/llama.cpp
/bin/bash: conda: command not found


In [102]:
!python convert-hf-to-gguf.py {finetuned}/llama-3-8b-chat-doctor-merged/ \
    --outfile {finetuned}/llama-3-8b-chat-doctor.gguf \
    --outtype f16

INFO:hf-to-gguf:Loading model: llama-3-8b-chat-doctor-merged
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Set model parameters
INFO:hf-to-gguf:gguf: context length = 8192
INFO:hf-to-gguf:gguf: embedding length = 4096
INFO:hf-to-gguf:gguf: feed forward length = 14336
INFO:hf-to-gguf:gguf: head count = 32
INFO:hf-to-gguf:gguf: key-value head count = 8
INFO:hf-to-gguf:gguf: rope theta = 500000.0
INFO:hf-to-gguf:gguf: rms norm epsilon = 1e-05
INFO:hf-to-gguf:gguf: file type = 1
INFO:hf-to-gguf:Set model tokenizer
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO:gguf.vocab:Adding 280147 merge(s).
INFO:gguf.vocab:Setting special token type bos to 128256
INFO:gguf.vocab:Setting special token type eos to 128257
INFO:gguf.vocab:Setting special token type pad to 128257
INFO:gguf.vocab:Setting chat_template to {% for message in messages %}{{'<|im_start|>' + message['role'] + '
' + mes

# 4. Quantizing the GGUF model

In [6]:
import os
os.getcwd()

'/fs01/home/ws_aabboud/finetuning-and-alignment/Deloitte/finetuned'

## Quantization

In [8]:
# %cd {finetuned}

! ./llama.cpp/llama-quantize llama-3-8b-chat-doctor.gguf llama-3-8b-chat-doctor-Q4_K_M.gguf Q4_K_M

/bin/bash: /llama.cpp/llama-quantize: No such file or directory


NameError: name 'os' is not defined

# 5. Using the Fine-Tuned Model Locally